# Mission 1-1
Q. 주어진 이미지 데이터의 파일 명에 기반하여 "이미지 ID" 수 기준으로 "성별 & 스타일" 통계치를 낸다.


통계를 내기에 앞서 Training 데이터에 중복된 "이미지 ID"가 존재하므로 전처리하여 제거하는 과정을 거쳤다.
(제거된 이미지 ID: 02958, 09717, 10028, 17802)

Validation 데이터 내에서는 중복된 데이터가 존재하지 않았다.

1. 폴더에서 이미지 파일 처리:
os.listdir(folder_path)로 지정된 폴더 내 모든 파일을 순회하며, 확장자가 .jpg인 파일만 처리합니다.
extract_info_from_filename을 호출하여 각 이미지 파일에서 추출된 정보를 data 리스트에 저장합니다.

2. DataFrame으로 변환:
data 리스트를 pd.DataFrame(data)로 변환하여 데이터프레임 df에 저장합니다. 이 데이터프레임에는 W_T, image_id, era, style, gender, filename 열이 포함됩니다.

3. 중복된 이미지 ID 확인 및 제거:
- df[df.duplicated(subset='image_id', keep=False)]로 중복된 image_id를 가진 항목을 필터링하여 duplicates 데이터프레임에 저장하고, 중복된 파일들의 목록을 출력합니다.
- 마지막에 df[df.duplicated(subset='image_id', keep=False) == False]로 중복되지 않은 데이터만을 필터링하여 반환합니다.

Mission 1 요구 사항:
중복된 image_id 확인 및 제거 작업은 중복 데이터를 처리해야 한다는 요구 사항을 만족합니다.

4. 학습 및 검증 데이터 폴더 설정:
train_folder = 'data/training_image'
valid_folder = 'data/validation_image'
- 이 부분은 학습 및 검증 이미지 데이터가 저장된 폴더 경로를 설정합니다. 이 경로를 기준으로 process_folder 함수에서 파일을 불러옵니다.

5. 학습 및 검증 데이터 처리:
train_df = process_folder(train_folder)
valid_df = process_folder(valid_folder)
- process_folder 함수를 호출하여 학습 및 검증 데이터 폴더의 이미지를 각각 train_df와 valid_df 데이터프레임으로 처리합니다. 이 데이터프레임에는 이미지 파일명에서 추출한 성별 및 스타일 정보가 포함됩니다.

6. 성별 및 스타일별 이미지 개수 세기:
train_counts = train_df.groupby(['gender', 'style']).size().reset_index(name='count')
valid_counts = valid_df.groupby(['gender', 'style']).size().reset_index(name='count')
- train_df와 valid_df에서 groupby(['gender', 'style'])로 성별 및 스타일 조합별로 그룹화하여 각 그룹에 속하는 이미지 개수를 셉니다.
size()는 각 그룹의 크기를 계산하며, reset_index(name='count')로 결과를 DataFrame 형태로 정리합니다.

Mission 1 요구 사항:
train_counts와 valid_counts는 성별 및 스타일별 이미지 수를 나타내는 통계표를 작성해야 한다는 요구 사항을 충족합니다.

7. 결과 출력:
print("\nTrain Image Counts by Gender and Style:")
print(train_counts)
print("\nValidation Image Counts by Gender and Style:")
print(valid_counts)
- 위에서 구한 train_counts와 valid_counts를 출력하여 학습 및 검증 데이터의 성별 및 스타일별 이미지 수 통계를 확인할 수 있습니다.

1. 클래스 조합 추출 및 정렬
- # 추출된 클래스 조합 확인 및 정렬
class_combinations = train_df[['gender', 'style']].drop_duplicates()
class_combinations = class_combinations.sort_values(['gender', 'style'], ascending=[False, True])
- class_combinations: 학습 데이터셋(train_df)에서 gender와 style의 조합을 중복 없이 추출합니다.
- 정렬: gender를 기준으로 내림차순, style을 기준으로 오름차순 정렬하여 순서를 일관되게 만듭니다. 이는 이후 클래스 ID 할당 시 순서를 명확히 하기 위함입니다.

Mission 1 요구 사항:
데이터에서 고유한 성별과 스타일 조합을 확인하고, 이를 통해 학습에 필요한 클래스 정보를 준비합니다.

2. 클래스 매핑 생성
- # 정렬된 클래스 조합을 기반으로 클래스 매핑 생성
class_mapping = {tuple(row): idx for idx, row in class_combinations.reset_index(drop=True).iterrows()}
print("\nClass mapping:")
print(class_mapping)
- class_mapping: 정렬된 class_combinations를 반복하며 gender, style 조합을 키로 하고 해당 순서 인덱스(idx)를 값으로 하는 딕셔너리를 생성합니다.
- 예를 들어, ('남성', 'hippie')가 0번 클래스라면, class_mapping에 {('남성', 'hippie'): 0}이 저장됩니다.
- 클래스 매핑 출력: class_mapping을 출력하여 각 조합의 클래스 ID를 확인할 수 있습니다.

Mission 1 요구 사항:
성별 및 스타일 조합을 고유한 클래스 ID로 변환하여 모델 학습 시 사용할 수 있도록 준비합니다.

3. 클래스 ID 할당
- # 클래스 ID 할당
train_df['class_id'] = train_df.apply(lambda row: class_mapping[(row['gender'], row['style'])], axis=1)
valid_df['class_id'] = valid_df.apply(lambda row: class_mapping[(row['gender'], row['style'])], axis=1)
- 학습(train_df)과 검증(valid_df) 데이터프레임에 class_id 열을 추가하여, 각 이미지에 대해 해당 성별 및 스타일 조합의 클래스 ID를 할당합니다.
- apply와 lambda를 사용하여 각 행의 gender와 style 조합을 class_mapping 딕셔너리에서 찾고, 이에 해당하는 클래스 ID를 할당합니다.

Mission 1 요구 사항:
- 각 이미지에 대해 고유한 클래스 ID를 부여하여, 학습 및 검증 데이터셋을 모델에 입력하기에 적합한 형태로 준비합니다.

4. 클래스 매핑 테이블 생성 및 출력
- # 클래스 매핑 테이블 생성
class_mapping_table = pd.DataFrame(list(class_mapping.keys()), columns=['Gender', 'Style'])
class_mapping_table['Class ID'] = list(class_mapping.values())
class_mapping_table = class_mapping_table[['Class ID', 'Gender', 'Style']]
print("\nClass Mapping Table:")
print(class_mapping_table)
- class_mapping 딕셔너리를 DataFrame으로 변환하여 Class ID, Gender, Style 열을 가지는 매핑 테이블을 만듭니다. 이 테이블은 각 클래스 ID가 어떤 성별 및 - 스타일 조합에 해당하는지 보여줍니다.
출력하여 각 클래스 ID와 그에 해당하는 성별, 스타일 조합을 확인할 수 있습니다.

Mission 1 요구 사항:
각 클래스 ID에 해당하는 성별과 스타일 조합을 명확히 정의하여 모델 해석 및 분석에 용이하도록 만듭니다.

5. 데이터 저장
- # CSV 파일로 저장
train_df.to_csv('data/train_metadata_with_class.csv', index=False)
valid_df.to_csv('data/valid_metadata_with_class.csv', index=False)
- train_df와 valid_df를 CSV 파일로 저장하여, 학습 및 검증 데이터셋에 대한 메타데이터 및 클래스 ID 정보를 보존합니다.
- train_metadata_with_class.csv 및 valid_metadata_with_class.csv 파일은 이후 분석 및 모델 학습 시 유용하게 활용할 수 있습니다.

6. GPU 사용 가능 여부 확인
import torch
# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 그래픽카드 정보 출력
if torch.cuda.is_available():
    print("\nGPU:")
    print(torch.cuda.get_device_name(device))
else:
    print("\nGPU: Not available")
- torch.cuda.is_available()로 GPU가 사용 가능한지 확인하고, 사용 가능한 경우 torch.device('cuda')로 GPU 장치를 설정합니다.
- GPU가 사용 가능한 경우 그래픽카드 이름을 출력하고, 사용 불가능한 경우 "GPU: Not available"을 출력합니다.

Mission 1 요구 사항:
학습 속도 최적화를 위해 GPU 사용 가능 여부를 확인하고, 가능하다면 GPU 장치에 모델을 할당할 준비를 합니다.




In [ ]:
import os
import pandas as pd

# 파일명에서 정보 추출하는 함수
def extract_info_from_filename(filename):
    parts = filename.split('_')
    # gender = '여성' if parts[4].split('.')[0] == 'W' else '남성'
    return {
        'W_T': parts[0],
        'image_id': parts[1],
        'era': parts[2],
        'style': parts[3],
        'gender': parts[4].split('.')[0],
        'filename': filename
    }

# 폴더 내 이미지 파일 처리하는 함수
def process_folder(folder_path):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg'):
            info = extract_info_from_filename(filename)
            data.append(info)
    # DataFrame으로 변환
    df = pd.DataFrame(data)

    # 중복된 image_id를 가진 항목들 확인
    duplicates = df[df.duplicated(subset='image_id', keep=False)]
    print(f"\n중복된 image_id 목록 ({folder_path}):")
    print(duplicates[['image_id', 'filename']].sort_values('image_id').to_string(index=False))

    # 중복된 데이터 제거 후 반환
    df = df[df.duplicated(subset='image_id', keep=False) == False]
    return df

# 트레인과 검증 폴더 경로 설정
train_folder = 'data/training_image'
valid_folder = 'data/validation_image'

# 트레인과 검증 폴더 처리
train_df = process_folder(train_folder)
valid_df = process_folder(valid_folder)

# Gender와 Style별 이미지 개수 세기
train_counts = train_df.groupby(['gender', 'style']).size().reset_index(name='count')
valid_counts = valid_df.groupby(['gender', 'style']).size().reset_index(name='count')

print("\nTrain Image Counts by Gender and Style:")
print(train_counts)

print("\nValidation Image Counts by Gender and Style:")
print(valid_counts)


중복된 image_id 목록 (data/training_image):
image_id                        filename
   02958       T_02958_19_normcore_M.jpg
   02958           W_02958_60_mods_M.jpg
   09717     T_09717_19_genderless_W.jpg
   09717       W_09717_19_normcore_W.jpg
   10028        W_10028_50_classic_W.jpg
   10028 T_10028_10_sportivecasual_W.jpg
   17802       T_17802_19_normcore_M.jpg
   17802           W_17802_80_bold_M.jpg

중복된 image_id 목록 (data/validation_image):
Empty DataFrame
Columns: [image_id, filename]
Index: []

Train Image Counts by Gender and Style:
   gender           style  count
0       M            bold    267
1       M          hiphop    274
2       M          hippie    260
3       M             ivy    237
4       M     metrosexual    278
5       M            mods    268
6       M        normcore    362
7       M  sportivecasual    298
8       W      athleisure     67
9       W   bodyconscious     95
10      W        cityglam     67
11      W         classic     76
12      W           dis

In [ ]:
# 추출된 클래스 조합 확인 및 정렬
class_combinations = train_df[['gender', 'style']].drop_duplicates()
class_combinations = class_combinations.sort_values(['gender', 'style'], ascending=[False, True])

# 정렬된 클래스 조합을 기반으로 클래스 매핑 생성
class_mapping = {tuple(row): idx for idx, row in class_combinations.reset_index(drop=True).iterrows()}
print("\nClass mapping:")
print(class_mapping)

# 클래스 ID 할당
train_df['class_id'] = train_df.apply(lambda row: class_mapping[(row['gender'], row['style'])], axis=1)
valid_df['class_id'] = valid_df.apply(lambda row: class_mapping[(row['gender'], row['style'])], axis=1)

# 클래스 매핑 테이블 생성
class_mapping_table = pd.DataFrame(list(class_mapping.keys()), columns=['Gender', 'Style'])
class_mapping_table['Class ID'] = list(class_mapping.values())
class_mapping_table = class_mapping_table[['Class ID', 'Gender', 'Style']]
print("\nClass Mapping Table:")
print(class_mapping_table)

# CSV 파일로 저장
train_df.to_csv('data/train_metadata_with_class.csv', index=False)
valid_df.to_csv('data/valid_metadata_with_class.csv', index=False)


Class mapping:
{('W', 'athleisure'): 0, ('W', 'bodyconscious'): 1, ('W', 'cityglam'): 2, ('W', 'classic'): 3, ('W', 'disco'): 4, ('W', 'ecology'): 5, ('W', 'feminine'): 6, ('W', 'genderless'): 7, ('W', 'grunge'): 8, ('W', 'hiphop'): 9, ('W', 'hippie'): 10, ('W', 'kitsch'): 11, ('W', 'lingerie'): 12, ('W', 'lounge'): 13, ('W', 'military'): 14, ('W', 'minimal'): 15, ('W', 'normcore'): 16, ('W', 'oriental'): 17, ('W', 'popart'): 18, ('W', 'powersuit'): 19, ('W', 'punk'): 20, ('W', 'space'): 21, ('W', 'sportivecasual'): 22, ('M', 'bold'): 23, ('M', 'hiphop'): 24, ('M', 'hippie'): 25, ('M', 'ivy'): 26, ('M', 'metrosexual'): 27, ('M', 'mods'): 28, ('M', 'normcore'): 29, ('M', 'sportivecasual'): 30}

Class Mapping Table:
    Class ID Gender           Style
0          0      W      athleisure
1          1      W   bodyconscious
2          2      W        cityglam
3          3      W         classic
4          4      W           disco
5          5      W         ecology
6          6      W    

In [ ]:
import torch
# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 그래픽카드 정보 출력
if torch.cuda.is_available():
    print("\nGPU:")
    print(torch.cuda.get_device_name(device))
else:
    print("\nGPU: Not available")


GPU: Not available


mission 1-2
1. Focal Loss 구현
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = (1 - pt) ** self.gamma * CE_loss

        if self.alpha is not None:
            alpha_factor = self.alpha.gather(0, targets)
            F_loss *= alpha_factor

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss
설명:
- Focal Loss는 클래스 불균형 문제를 완화하기 위한 손실 함수입니다. 잘못 분류된 샘플에 더 큰 가중치를 부여해 모델이 어려운 샘플에 더 집중하도록 합니다.
- 파라미터: gamma는 손실의 형태를 조정하며, alpha는 특정 클래스에 가중치를 줄 때 사용합니다.

Mission 1-2 요구 사항:
Focal Loss는 불균형한 데이터셋에 대해 성능을 향상시킬 수 있습니다. 따라서 성능 향상을 위해 선택된 손실 함수입니다.

2. ResNet-18 모델 아키텍처 구현
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        # 3x3 Conv 레이어 및 BatchNorm 설정
        ...
    def forward(self, x):
        ...
        return out

class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18, self).__init__()
        ...
    def _make_layer(self, out_channels, num_blocks, stride):
        ...
    def forward(self, x):
        ...
        return out
설명:
- ResidualBlock: ResNet-18의 기본 단위로, 각 블록 내에는 두 개의 3x3 합성곱 레이어와 Batch Normalization 및 ReLU 활성화 함수가 사용됩니다. 입력과 출력을 더하는 **스킵 연결(shortcut connection)**이 포함되어 있어 기울기 소실 문제를 해결합니다.
- ResNet-18: 입력 레이어, 여러 블록으로 이루어진 네 개의 레이어, 풀링 레이어 및 최종 분류 레이어로 구성됩니다.
- num_classes 파라미터는 분류해야 하는 클래스 수입니다. 이 문제에서 클래스 수는 성별과 스타일 조합 수와 동일합니다.

Mission 1-2 요구 사항:
- ResNet-18 구조를 무작위 초기화된 가중치로 구현함으로써 전이 학습 없이 모델을 구성하고 학습할 수 있도록 했습니다.

3. 데이터셋 클래스 정의 및 전처리
class FashionDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        ...
    def __len__(self):
        ...
    def __getitem__(self, idx):
        ...
        return image, label
설명:
- FashionDataset 클래스는 PyTorch의 Dataset을 상속받아 작성된 사용자 정의 데이터셋 클래스입니다.
- __getitem__ 함수는 csv_file에서 filename을 통해 이미지를 불러오고, transform을 적용한 뒤 이미지와 클래스 ID(class_id)를 반환합니다.

Mission 1-2 요구 사항:
- 학습 데이터와 검증 데이터를 PyTorch에서 사용할 수 있도록 사용자 정의 데이터셋 클래스를 통해 구성합니다.

4. Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        ...
    def __call__(self, val_loss, model):
        ...
설명:
- EarlyStopping은 검증 손실이 개선되지 않으면 학습을 멈추는 기능으로, 과적합(overfitting)을 방지할 수 있습니다. patience는 학습을 중단하기 전까지 허용되는 에폭 수입니다.

Mission 1-2 요구 사항:
- Early Stopping은 검증 손실을 기반으로 학습을 조기 종료하여 과적합을 방지하므로 성능 향상에 기여할 수 있습니다.

5. 데이터 전처리 설정
def get_transforms(augmentations=[]):
    transform_list = [transforms.Resize((224, 224)), transforms.ToTensor(),
                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
    ...
    return transforms.Compose(transform_list)

설명:
get_transforms 함수는 기본 이미지 크기 조정 및 정규화를 수행하며, 학습 데이터에서는 수평 뒤집기 및 블러와 같은 데이터 증강을 추가하여 모델의 일반화 성능을 높입니다.

6. 데이터셋 및 데이터로더 설정
train_transform = get_transforms(augmentations=['horizontal_flip', 'gaussian_blur'])
train_dataset = FashionDataset(csv_file='train_metadata_with_class.csv',
                               root_dir=train_folder,
                               transform=train_transform)

valid_dataset = FashionDataset(csv_file='valid_metadata_with_class.csv',
                               root_dir=valid_folder,
                               transform=get_transforms())  # 검증 셋은 증강 없이

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32)

설명:
학습용 데이터에서는 데이터 증강을 적용하여 train_loader에 데이터셋을 로드하며, 검증용 데이터는 증강 없이 valid_loader로 로드합니다.

7. 모델, 옵티마이저, 스케줄러 및 손실 함수 설정
model = ResNet18(num_classes=len(class_mapping)).to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
criterion = FocalLoss(gamma=2)

설명:
AdamW 옵티마이저는 가중치 감쇠(weight decay)를 추가하여 과적합을 방지하고, 학습률 스케줄러 ReduceLROnPlateau는 검증 손실이 개선되지 않으면 학습률을 조정합니다.

8. 학습 및 검증 루프
for epoch in range(num_epochs):
    model.train()
    ...
    
    # 검증 루프
    model.eval()
    ...
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.2f}%')
    
    scheduler.step(val_loss)
    early_stopping(val_loss, model)
    
    if early_stopping.early_stop:
        print("Early stopping")
        break
설명:
각 에폭에서 모델을 학습하고, 검증 손실과 정확도를 출력합니다.
검증 후 scheduler.step과 early_stopping을 통해 학습률 조정 및 조기 종료 조건을 확인합니다.




In [ ]:
# Mission 1-2

import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Focal Loss 구현
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = (1 - pt) ** self.gamma * CE_loss

        if self.alpha is not None:
            alpha_factor = self.alpha.gather(0, targets)
            F_loss *= alpha_factor

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

# ResNet-18 아키텍처
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Identity()

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(residual)
        out = self.relu(out)
        return out

class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(ResidualBlock(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

# 데이터셋 클래스 정의
class FashionDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.root_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')
        label = row['class_id']

        if self.transform:
            image = self.transform(image)

        return image, label

# Early Stopping 클래스 정의
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.delta = delta

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# 전처리 적용 함수
def get_transforms(augmentations=[]):
    transform_list = [transforms.Resize((224, 224)), transforms.ToTensor(),
                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]

    # augmentations 목록에 따라 전처리 추가
    if 'horizontal_flip' in augmentations:
        transform_list.insert(0, transforms.RandomHorizontalFlip())
    if 'gaussian_blur' in augmentations:
        transform_list.insert(0, transforms.GaussianBlur(kernel_size=3))
    if 'random_grayscale' in augmentations:
        transform_list.insert(0, transforms.RandomGrayscale(p=0.2))

    return transforms.Compose(transform_list)

# 데이터셋 & 데이터로더 설정
train_transform = get_transforms(augmentations=['horizontal_flip', 'gaussian_blur'])
train_dataset = FashionDataset(csv_file='train_metadata_with_class.csv',
                               root_dir=train_folder,
                               transform=train_transform)

valid_dataset = FashionDataset(csv_file='valid_metadata_with_class.csv',
                               root_dir=valid_folder,
                               transform=get_transforms())  # 검증 셋은 증강 없이

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32)

# 모델, 옵티마이저, 스케줄러, 손실 함수 설정
model = ResNet18(num_classes=len(class_mapping)).to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
criterion = FocalLoss(gamma=2)

# Early Stopping 인스턴스
early_stopping = EarlyStopping(patience=7)

# 학습 루프
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # 검증 루프
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(valid_loader)
    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.2f}%')

    # 스케줄러와 Early Stopping 확인
    scheduler.step(val_loss)
    early_stopping(val_loss, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break


GPU: Not available
